# Получение и обработка чеков из ОФД

In [1]:
import requests
import pandas as pd
import time
from datetime import datetime as dt

API ofd.ru для запроса требует ключ api, ИНН организации, регистрационный и серийный номер кассового аппарата

In [2]:
APIKEY = "10d3be6b4c214485a5854c8ec474716e"
URL = "https://ofd.ru/api/integration/v2/inn/{}/kkt/{}/receipts-with-fpd-short"
INN = "8904091366"
KKTSN = "00106901639781"
KKTRNM = "0005984435024549"

In [18]:
def gas_name(item_name: str):
    res = ""
    if item_name.find("аи-92-к5"):
        res = "92"
    elif item_name.find("аи-95-к5"):
        res = "95"
    elif item_name.find("аи-98-к5"):
        res = "98"
    else:
        res = "дт"
    return res

In [4]:
years = ['2023', '2024', '2025']
params = {
    "dateFrom": dt(2023, 1, 1, 0, 0, 0, 0).isoformat(),
    "dateTo": dt(2023, 1, 31, 23, 59, 59, 0).isoformat(),
    "AuthToken": APIKEY,
}

In [5]:
response = requests.get(URL.format(INN, KKTRNM), params).json()
time.sleep(1)
display(response)
# ofd_df = pd.DataFrame(columns=["year", "month", "day", "gas", "quantity", "price", "income"])

{'Status': 'Success',
 'Data': [{'Id': '0016d07d-34fc-42f1-7a1b-aed18df57d46',
   'CDateUtc': '2023-01-19T19:32:09',
   'Tag': 3,
   'IsBso': False,
   'IsCorrection': False,
   'OperationType': 'Income',
   'TaxationType': 1,
   'UserInn': '8904091366',
   'KktRegNumber': '0005984435024549',
   'FnNumber': '9960440300783687',
   'DocNumber': 63374,
   'DocDateTime': '2023-01-19T21:32:00',
   'Operator': 'Ильичева В.Г.',
   'DocShiftNumber': 476,
   'DecimalFiscalSign': '953554741',
   'ReceiptNumber': '160',
   'DocRawId': '0016d07d-34fc-42f1-7a1b-aed18df57d46',
   'Items': [{'Name': 'аи-92-к5 92 (2 трк)',
     'Price': 4820,
     'Quantity': 20.75,
     'Total': 100015,
     'CalculationMethod': 4,
     'SubjectType': 1,
     'NDS_Summ': 16669,
     'NDS_Rate': 1}],
   'Nds00_TotalSumm': 0,
   'NdsNA_TotalSumm': 0,
   'TotalSumm': 100000,
   'CashSumm': 0,
   'ECashSumm': 100000,
   'PrepaidSumm': 0,
   'CreditSumm': 0,
   'ProvisionSumm': 0,
   'TaxTotalSumm': 16669,
   'Tax10Summ':

In [21]:
sales = []
for reciept in response["Data"]:
    reciept_date = pd.to_datetime(reciept["DocDateTime"])
    for item in reciept["Items"]:
        sale = {}
        sale["year"] = reciept_date.year
        sale["month"] = reciept_date.month
        sale["day"] = reciept_date.day
        sale["gas"] = gas_name(item["Name"])
        sale["quantity"] = pd.to_numeric(item["Quantity"])
        sale["price"] = pd.to_numeric(item["Price"])
        sale["income"] = pd.to_numeric(item["Total"])
        sales.append(sale)

ofd_df = pd.DataFrame(sales)
#sorted_ofd_df = ofd_df.sort_values(by=['year', 'month', 'day', 'gas'])
sg_ofd_df = ofd_df.groupby(by=['year', 'month', 'day', 'gas']).sum().sort_values(by=['year', 'month', 'day', 'gas']).reset_index()
display(sg_ofd_df)

,year,month,day,gas,quantity,price,income
0,2023,1,1,92,467.42,175150,2565458
1,2023,1,1,95,750.55,144600,3617654
2,2023,1,2,92,1113.89,299250,5921431
3,2023,1,2,95,984.83,202440,4746885
4,2023,1,3,92,1172.70,262950,6276347
...,...,...,...,...,...,...,...
57,2023,1,29,95,1526.10,294020,7355808
58,2023,1,30,92,2119.73,382250,11408026
59,2023,1,30,95,1784.77,327760,8602594
60,2023,1,31,92,2261.33,392040,12207819
